# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


In [3]:
file_path_list[:5]

['/home/workspace/event_data/2018-11-05-events.csv',
 '/home/workspace/event_data/2018-11-23-events.csv',
 '/home/workspace/event_data/2018-11-22-events.csv',
 '/home/workspace/event_data/2018-11-21-events.csv',
 '/home/workspace/event_data/2018-11-03-events.csv']

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
print('total rows in full_data_rows_list: ' + str(len(full_data_rows_list)))

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


total rows in full_data_rows_list: 8056


In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [6]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [7]:
# Create a Keyspace 
session.execute(""" CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION = 
    {'class':'SimpleStrategy', 'replication_factor':1}""")

#### Set Keyspace

In [8]:
# Set KEYSPACE to the keyspace specified above
session.set_keyspace('udacity')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### Table 1: session_playlist
- **Partition key(s):** sessionId (int)
- **Clustering key(s):** itemInSession (int)
- artist (text)
- song (text)
- length (float)

The query specifies that it will need the artist, song, and length of an entry corresponding to a sessionId and item. I have opted to partition the data by sessionId and cluster it by itemInSession because we need to use those keys in the WHERE clause, and together they will comprise a unique song. It does not make sense to bother partitioning by itemInSession.

In [9]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

q1 = """ CREATE TABLE IF NOT EXISTS session_playlist
    (sessionId int, itemInSession int, artist text, song text, length float, PRIMARY KEY (sessionId, itemInSession))"""
session.execute(q1)                    

In [10]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # Assign the INSERT statements into the `query` variable
        query = "INSERT INTO session_playlist (sessionId, itemInSession, artist, song, length) "
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[]), int(line[]), line[], line[], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [11]:
# Add in the SELECT statement to verify the data was entered into the table
q = """SELECT artist, song, length FROM session_playlist WHERE sessionId = 338 AND itemInSession = 4"""
try:
    rows = session.execute(q)
except Exception as e:
    print(e)
    
for r in rows:
    print(r.artist, r.song, r.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### Table 2: user_session
- **Partition key(s):** userId (int), sessionId (int)
- **Clustering key(s):** itemInSession (int)
- artist (text)
- song (text)
- firstName (text)
- lastName(text)

The query specifies that it will need the song and full name of an entry corresponding to a userId and sessionId and item; it is further specified that the table shall be ordered by sessionId. I have opted to partition the data by userId and sessionId, and cluster it by itemInSession. This way, each user session will be stored on the same node. I can use sessionId as a clustering column to achieve the correct order, and do not need to bother using it as a partition key.

In [12]:
# Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
# for userid = 10, sessionid = 182
q2 = """ CREATE TABLE IF NOT EXISTS user_session
    (userId int, sessionId int, itemInSession int, artist text, song text, firstName text, lastName text,
    PRIMARY KEY ((userId, sessionId), itemInSession))"""
session.execute(q2) 

In [13]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # Assign the INSERT statements into the `query` variable
        query = "INSERT INTO user_session (userId, sessionId, itemInSession, artist, song, firstName, lastName) "
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [14]:
q = """SELECT artist, song, firstName, lastName FROM user_session WHERE userId = 10 AND sessionId = 182"""
try:
    rows = session.execute(q)
except Exception as e:
    print(e)
    
for r in rows:
    print(r.artist, r.song, r.firstname, r.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### Table 3: song_fans
- **Partition key(s):** song (text)
- **Clustering key(s):** userId (int)
- firstName (text)
- lastName(text)

The query specifies that it will need the full name of all users who have listened to a particular song. I opted to use the song as the partition key and the userId as the clustering key because they will always have a unique pairing, and this method will store all users who have listened to a given song on the same node. 

In [15]:
# Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
q3 = """ CREATE TABLE IF NOT EXISTS song_fans
    (song text, userId int, firstName text, lastName text,
    PRIMARY KEY (song, userId))"""
session.execute(q3)

In [17]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_fans (song, userId, firstName, lastName) "
        query = query + "VALUES (%s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [18]:
q = """SELECT firstName, lastName FROM song_fans WHERE song = 'All Hands Against His Own'"""
try:
    rows = session.execute(q)
except Exception as e:
    print(e)
    
for r in rows:
    print(r.firstname, r.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [19]:
query = "drop table session_playlist"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table user_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table song_fans"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [20]:
session.shutdown()
cluster.shutdown()